In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cervical_Cancer"
cohort = "GSE138080"

# Input paths
in_trait_dir = "../../input/GEO/Cervical_Cancer"
in_cohort_dir = "../../input/GEO/Cervical_Cancer/GSE138080"

# Output paths
out_data_file = "../../output/preprocess/Cervical_Cancer/GSE138080.csv"
out_gene_data_file = "../../output/preprocess/Cervical_Cancer/gene_data/GSE138080.csv"
out_clinical_data_file = "../../output/preprocess/Cervical_Cancer/clinical_data/GSE138080.csv"
json_path = "../../output/preprocess/Cervical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identification of deregulated pathways, key regulators, and novel miRNA-mRNA interactions in HPV-mediated transformation. [mRNA tissues-Agilent]"
!Series_summary	"Next to a persistent infection with high-risk human papillomavirus (HPV), molecular changes are required for the development of cervical cancer. To identify which molecular alterations drive carcinogenesis, we performed a comprehensive and longitudinal molecular characterization of HPV-transformed keratinocyte cell lines. Comparative genomic hybridization, mRNA, and miRNA expression analysis of four HPV-containing keratinocyte cell lines at eight different time points was performed. Data was analyzed using unsupervised hierarchical clustering, integrated longitudinal expression analysis, and pathway enrichment analysis. Biological relevance of identified key regulatory genes was evaluated in vitro and dual-luciferase assays were used to confirm predicted miRNA-mRNA interactions. We show 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import numpy as np
from typing import Optional, Callable, List, Dict, Any, Union
import json

# 1. Gene Expression Data Assessment
# Based on the background information and series title, this appears to be mRNA expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Examining the Sample Characteristics Dictionary
# The first key (0) contains information about cell types: normal, CIN2/3, and squamous cell carcinoma
# The second key (1) contains HPV status: high-risk HPV-positive or HPV-negative

trait_row = 0  # Cell type information can be used to determine cervical cancer
age_row = None  # No age information available
gender_row = None  # No gender information available (though we can assume all samples are female, but it's not explicit)

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> Optional[int]:
    """
    Convert cell type information to binary trait data for cervical cancer.
    0: normal tissue (control)
    1: cancer or precancerous lesion (case)
    """
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    
    if 'normal' in value:
        return 0
    elif 'carcinoma' in value or 'neoplasia' in value:
        return 1
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """
    Convert age information to continuous value.
    Not used in this dataset as age information is not available.
    """
    return None

def convert_gender(value: str) -> Optional[int]:
    """
    Convert gender information to binary.
    Not used in this dataset as gender information is not available.
    """
    return None

# 3. Save Metadata
# Perform initial filtering and save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Check if trait_row is not None (meaning clinical data is available)
if trait_row is not None:
    # We need to check if clinical_data is available in the environment
    # This should have been loaded in a previous step
    try:
        # Extract clinical features using the library function
        clinical_features_df = geo_select_clinical_features(
            clinical_df=clinical_data,  # Assuming clinical_data was loaded in previous step
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        print("Preview of extracted clinical features:")
        preview = preview_df(clinical_features_df)
        print(preview)
        
        # Create output directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical features to CSV
        clinical_features_df.to_csv(out_clinical_data_file, index=True)
        print(f"Clinical features saved to {out_clinical_data_file}")
    except NameError:
        print("Error: clinical_data not found. Please ensure it was loaded in a previous step.")


Preview of extracted clinical features:
{'GSM4098861': [0.0], 'GSM4098862': [0.0], 'GSM4098863': [0.0], 'GSM4098864': [0.0], 'GSM4098865': [0.0], 'GSM4098866': [0.0], 'GSM4098867': [0.0], 'GSM4098868': [0.0], 'GSM4098869': [0.0], 'GSM4098870': [0.0], 'GSM4098871': [1.0], 'GSM4098872': [1.0], 'GSM4098873': [1.0], 'GSM4098874': [1.0], 'GSM4098875': [1.0], 'GSM4098876': [1.0], 'GSM4098877': [1.0], 'GSM4098878': [1.0], 'GSM4098879': [1.0], 'GSM4098880': [1.0], 'GSM4098881': [1.0], 'GSM4098882': [1.0], 'GSM4098883': [1.0], 'GSM4098884': [1.0], 'GSM4098885': [1.0], 'GSM4098886': [1.0], 'GSM4098887': [1.0], 'GSM4098888': [1.0], 'GSM4098889': [1.0], 'GSM4098890': [1.0], 'GSM4098891': [1.0], 'GSM4098892': [1.0], 'GSM4098893': [1.0], 'GSM4098894': [1.0], 'GSM4098895': [1.0]}
Clinical features saved to ../../output/preprocess/Cervical_Cancer/clinical_data/GSE138080.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['12', '14', '16', '17', '19', '22', '24', '25', '27', '30', '33', '35',
       '37', '38', '40', '42', '45', '47', '49', '51'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers appear to be numeric values (likely probe IDs or some other non-gene symbol identifiers)
# These are not standard human gene symbols which are typically alphanumeric (e.g., TP53, BRCA1, etc.)
# Therefore, mapping to gene symbols would be required

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['266', '266', '266', '266', '266'], 'ROW': [170.0, 168.0, 166.0, 164.0, 162.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner', 'DarkCorner'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan], 'SPOT_ID.1': [nan, nan, nan, nan, nan], 'ORDER': [1.0, 2.0, 3.

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate columns for mapping
# Based on observation: 'ID' column in gene_annotation contains probe IDs
# and 'GENE_SYMBOL' column contains the gene symbols

# 2. Get a gene mapping dataframe by extracting the relevant columns 
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# 3. Apply the gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print the first few gene symbols to verify the mapping worked
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])


First 10 gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'AAAS',
       'AACS', 'AADACL1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Load the clinical features from the previously saved file (reusing the data from Step 2)
clinical_features_df = pd.read_csv(out_clinical_data_file, index_col=0)

# Now link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data comparing normal cervical tissue (n=10) to CIN2/3 lesions (n=15) and cervical squamous cell carcinomas (n=10)."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Cervical_Cancer/gene_data/GSE138080.csv
Linked data shape: (35, 13053)


For the feature 'Cervical_Cancer', the least common label is '0.0' with 10 occurrences. This represents 28.57% of the dataset.
The distribution of the feature 'Cervical_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Cervical_Cancer/GSE138080.csv
